<a href="https://colab.research.google.com/github/sanjaya999/Pytorch/blob/main/bike_or_scooter_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#get data
import torch
from torch import nn

device = "cuda" if torch.cuda.is_available() else "cpu"
device
import requests
import zipfile
from pathlib import Path

data_path = Path("data/")
image_path = data_path

#if image folder doent exist , download it and prepare

if image_path.is_dir():
  print(f"{image_path} directory already exist")
else:
  print(f"creating {image_path} directory")
  image_path.mkdir(parents=True,exist_ok=True)

#download test file
with open(data_path / "data2.zip" , "wb") as f:
  request= requests.get("https://github.com/sanjaya999/Pytorch/raw/refs/heads/main/data2.zip")
  print("downloading bike scooter image")
  f.write(request.content)

#unzip
with zipfile.ZipFile(data_path/ "data2.zip" , "r") as zip_ref:
  print("unzipping bikeScooter")
  zip_ref.extractall(image_path)


creating data directory
downloading bike scooter image
unzipping bikeScooter


In [2]:
train_dir='/content/data/data2/train/'
test_dir ='/content/data/data2/test/'

In [3]:
from PIL import Image, UnidentifiedImageError
import os

def clean_bad_images(folder):
    for subdir, _, files in os.walk(folder):
        for file in files:
            filepath = os.path.join(subdir, file)
            try:
                with Image.open(filepath) as img:
                    img.verify()
            except (UnidentifiedImageError, OSError):
                print(f"Removing corrupted file: {filepath}")
                os.remove(filepath)

# Clean your dataset
clean_bad_images(train_dir)
clean_bad_images(test_dir)

Removing corrupted file: /content/data/data2/train/bike/Bajaj Bikes in India  Price, New Models 2025, Mile_4.jpg
Removing corrupted file: /content/data/data2/train/bike/KTM Bikes Price - Check Images, Reviews & Specs in_1.jpg
Removing corrupted file: /content/data/data2/train/bike/Quality, High-Performance suzuki bike price - Alib.jpg
Removing corrupted file: /content/data/data2/train/bike/Yamaha Bikes - Latest Price List 2025, New Models _5.jpg
Removing corrupted file: /content/data/data2/train/bike/TVS Bikes Price in India - New TVS Models 2025, Im_3.jpg
Removing corrupted file: /content/data/data2/test/bike/KTM Bikes Price (March Offers!), Images, Reviews a.png
Removing corrupted file: /content/data/data2/test/bike/Yamaha Bikes in India  Price, New Models 2025, Mil_4.jpg


In [4]:
##transform data into tensors
import torch
from torch.utils.data import DataLoader
from torchvision import datasets , transforms

#transforming data with torchvision.transforms
#write a transform for image

data_transform = transforms.Compose([
    #resize image to 64x64
    transforms.Resize(size =(64,64)),

    #flip the image horizontally 50% of the time
    transforms.RandomHorizontalFlip(p=0.5),

    #convert image to tensor
    transforms.ToTensor()

])

In [5]:
#use image folder to create datasets

from torchvision import datasets
train_data = datasets.ImageFolder(root = train_dir,
                                 transform = data_transform
                                )

test_data = datasets.ImageFolder(root = test_dir,
                                transform = data_transform
                                )

test_data,train_data


(Dataset ImageFolder
     Number of datapoints: 27
     Root location: /content/data/data2/test/
     StandardTransform
 Transform: Compose(
                Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
            ),
 Dataset ImageFolder
     Number of datapoints: 272
     Root location: /content/data/data2/train/
     StandardTransform
 Transform: Compose(
                Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
            ))

In [6]:
#get class name as list
classname = train_data.classes
classname_dict = train_data.class_to_idx
print(classname)
print(classname_dict)


['bike', 'scooter']
{'bike': 0, 'scooter': 1}


In [8]:
import os
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset=train_data,
                              batch_size= 32,
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_data,
                              batch_size= 32,
                              shuffle=False)
print(len(train_data) , len(test_data))

train_dataloader , test_dataloader
#

272 27


(<torch.utils.data.dataloader.DataLoader at 0x79f25e4e8510>,
 <torch.utils.data.dataloader.DataLoader at 0x79f267bc4190>)

In [9]:
for batch, (X, y) in enumerate(test_dataloader):
  print(batch, X[0].shape, y)
  break

0 torch.Size([3, 64, 64]) tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1])


In [10]:
for batch, (X, y) in enumerate(train_dataloader):
  print(batch, X[0].shape, y)
  break

0 torch.Size([3, 64, 64]) tensor([1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
        1, 0, 1, 0, 0, 1, 0, 0])


In [11]:
# list(train_dataloader_simple)

In [12]:
simple_transform = transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])

In [13]:
train_dir

'/content/data/data2/train/'

In [14]:
#load and transform data

train_data_simple = datasets.ImageFolder(root=train_dir,
                                         transform=simple_transform,
                                         )
test_data_simple = datasets.ImageFolder(root=test_dir, transform=simple_transform,)

#setup batch size and number of worker

BATCH_SIZE = 32
NUM_WORKERS =os.cpu_count()

train_dataloader_simple = DataLoader(dataset=train_data_simple,
                                     batch_size=BATCH_SIZE,
                                     num_workers=NUM_WORKERS,
                                     shuffle=True)

test_dataloader_simple = DataLoader(dataset=test_data_simple,
                                    batch_size=BATCH_SIZE,
                                    num_workers=NUM_WORKERS,
                                    shuffle=False)

train_dataloader_simple , test_dataloader_simple

(<torch.utils.data.dataloader.DataLoader at 0x79f25e4e59d0>,
 <torch.utils.data.dataloader.DataLoader at 0x79f25e535950>)

In [15]:
#creating a model

class TinyVGG(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=1),
         nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                      stride=2)
    )
    self.conv_block_2= nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=1),
         nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                      stride=2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*16*16 ,out_features=output_shape),
        # nn.Sigmoid()
    )

  def forward(self, x: torch.Tensor):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x


In [16]:
model_0 = TinyVGG(input_shape=3,
                  hidden_units=3,
                  output_shape=2)
model_0

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=768, out_features=2, bias=True)
  )
)

In [17]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer):
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)
        # print(X.shape, y.shape)

        # 1. Forward pass
        y_pred = model(X)
        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metrics across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        # print(y_pred_class)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    # print(train_loss, train_acc)
    return train_loss, train_acc

In [21]:
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module):
    # Put model in eval mode
    model.eval()

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager

    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device##
            # print(X, y)
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

In [26]:
from tqdm.auto import tqdm

# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5):

    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }

    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        test_loss, test_acc = test_step(model=model, dataloader=test_dataloader,loss_fn=loss_fn)

        # 4. Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # 5. Update results dictionary
        # Ensure all data is moved to CPU and converted to float for storage
        results["train_loss"].append(train_loss.item() if isinstance(train_loss, torch.Tensor) else train_loss)
        results["train_acc"].append(train_acc.item() if isinstance(train_acc, torch.Tensor) else train_acc)
        results["test_loss"].append(test_loss.item() if isinstance(test_loss, torch.Tensor) else test_loss)
        results["test_acc"].append(test_acc.item() if isinstance(test_acc, torch.Tensor) else test_acc)

    # 6. Return the filled results at the end of the epochs
    return results

  # Setup loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=0.001)

model_0_results = train(model=model_0,
                        train_dataloader=train_dataloader_simple,
                        test_dataloader=test_dataloader_simple,
                        optimizer=optimizer,
                        loss_fn=loss_fn,
                        epochs=10)

  0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch: 1 | train_loss: 0.6781 | train_acc: 0.5868 | test_loss: 0.7120 | test_acc: 0.4815


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch: 2 | train_loss: 0.6827 | train_acc: 0.5729 | test_loss: 0.7124 | test_acc: 0.4815


...

Epoch: 8 | train_loss: 0.6792 | train_acc: 0.5833 | test_loss: 0.7124 | test_acc: 0.4815


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch: 9 | train_loss: 0.6804 | train_acc: 0.5799 | test_loss: 0.7125 | test_acc: 0.4815


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch: 10 | train_loss: 0.6747 | train_acc: 0.5972 | test_loss: 0.7124 | test_acc: 0.4815
